In [1]:
!pip install python-binance
!pip install catboost
!pip install matplotlib --upgrade
!pip install mpl_finance
!pip install --upgrade mplfinance

Requirement already up-to-date: matplotlib in c:\users\ampbo\anaconda3\lib\site-packages (3.4.1)
Requirement already up-to-date: mplfinance in c:\users\ampbo\anaconda3\lib\site-packages (0.12.7a17)


In [2]:
import pandas as pd
import numpy as np

import time
import dateparser
import pytz
import json

import math  

import datetime as dt
from datetime import datetime, timedelta

from threading import Thread

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates

from mpl_finance import candlestick_ochl



from sklearn import preprocessing, model_selection, neighbors, svm
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

from catboost import CatBoostClassifier, CatBoostRegressor, Pool, cv

from tqdm import tqdm as tqdm

from binance.client import Client

import datetime as dtt

C:\Users\ampbo\anaconda3\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [7]:
tipoint = pd.read_csv('tiposinteresok.csv')

In [8]:
tipoint 

,DATE,INTDSRUSM193N
0,2015-10-01,0.75
1,2015-11-01,0.75
2,2015-12-01,1.00
3,2016-01-01,1.00
4,2016-02-01,1.00
...,...,...
59,2020-09-01,0.25
60,2020-10-01,0.25
61,2020-11-01,0.25
62,2020-12-01,0.25


In [9]:
tipoint.describe()

,INTDSRUSM193N
count,64.000000
mean,1.625000
std,0.929029
min,0.250000
25%,1.000000
50%,1.750000
75%,2.312500
max,3.000000


In [10]:
tipoint.dtypes

DATE              object
INTDSRUSM193N    float64
dtype: object

In [13]:
tipoint['DATE']

0     2015-10-01
1     2015-11-01
2     2015-12-01
3     2016-01-01
4     2016-02-01
         ...    
59    2020-09-01
60    2020-10-01
61    2020-11-01
62    2020-12-01
63    2021-01-01
Name: DATE, Length: 64, dtype: object

In [14]:
tipoint['Open Time'] = pd.to_datetime(tipoint['DATE'])

In [16]:
print(tipoint)

          DATE  INTDSRUSM193N  Open Time
0   2015-10-01           0.75 2015-10-01
1   2015-11-01           0.75 2015-11-01
2   2015-12-01           1.00 2015-12-01
3   2016-01-01           1.00 2016-01-01
4   2016-02-01           1.00 2016-02-01
..         ...            ...        ...
59  2020-09-01           0.25 2020-09-01
60  2020-10-01           0.25 2020-10-01
61  2020-11-01           0.25 2020-11-01
62  2020-12-01           0.25 2020-12-01
63  2021-01-01           0.25 2021-01-01

[64 rows x 3 columns]


In [ ]:
sp500['Open Time'] = pd.to_datetime(sp500['Fecha'])
sp500['OpenSP'] = sp500['Apertura']
sp500['CloseSP'] = sp500['Último']

In [ ]:
df_with_sp500 = df.merge(sp500[["OpenSP", "CloseSP", "Open Time"]], on="Open Time")

In [ ]:
df_with_sp500.tail()

In [20]:
df = pd.read_csv('df_indexes_gold_eth.csv', index_col=0)

In [21]:
df

,Open Time,Open,High,Low,Close_btc,Volume,NumberTransactions,Close SP500,Open SP500,halving,Close Gold,Open Gold,OpenDJ,CloseDJ,Close_eth
0,2015-10-11,246.30,249.50,246.30,249.50,3021.120000,104550.0,2081.72,2077.19,1,1149.4,1149.4,17724.13,17758.21,0.60
1,2015-10-12,249.50,249.50,247.60,247.60,9493.890000,134410.0,2052.23,2047.93,1,1130.3,1130.3,17493.17,17574.75,0.63
2,2015-10-13,247.60,252.87,245.75,250.00,12875.400000,134878.0,2003.69,2015.00,1,1231.7,1231.7,17113.55,17081.89,0.60
3,2015-10-14,250.00,254.75,250.00,253.00,30115.770000,141985.0,1994.24,2003.66,1,1243.6,1243.6,17079.08,16924.75,0.50
4,2015-10-15,253.00,258.98,253.00,255.26,50535.450000,153495.0,2023.86,1996.47,1,1251.4,1251.4,16944.86,17141.75,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,2021-04-16,63158.74,63520.61,60000.00,61334.80,91764.139884,2639940.0,4185.47,4174.14,3,1779.0,1770.4,34058.16,34200.67,2422.67
2015,2021-04-17,61334.81,62506.05,59580.91,60006.66,58912.256128,2268816.0,4185.47,4174.14,3,1779.0,1770.4,34058.16,34200.67,2317.60
2016,2021-04-18,60006.67,60499.00,50931.30,56150.01,124882.131824,3444089.0,4185.47,4174.14,3,1779.0,1770.4,34058.16,34200.67,2235.64
2017,2021-04-19,56150.01,57526.81,54221.58,55633.14,78229.042267,2390527.0,4162.80,4175.60,3,1769.4,1782.1,34182.38,34077.63,2161.12


In [22]:
df.describe()

,Open,High,Low,Close_btc,Volume,NumberTransactions,Close SP500,Open SP500,halving,Close Gold,Open Gold,OpenDJ,CloseDJ,Close_eth
count,2019.000000,2019.000000,2019.000000,2019.000000,2.019000e+03,2.019000e+03,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000
mean,8118.617256,8373.885953,7845.329911,8146.388430,1.738177e+07,4.597203e+05,2722.603843,2723.500594,1.340763,1448.567221,1448.691441,23792.057499,23786.013418,307.488821
std,10787.222476,11161.188328,10387.891407,10839.617374,3.437011e+07,5.251333e+05,506.494218,507.698068,0.752121,210.520585,210.443203,4162.136936,4158.173286,402.850086
min,246.300000,249.500000,236.000000,247.600000,0.000000e+00,1.045500e+05,1829.080000,1833.400000,1.000000,1070.800000,1070.100000,15691.620000,15660.180000,0.420000
25%,1049.880000,1082.560000,1022.655000,1054.860000,3.166145e+05,2.237780e+05,2341.890000,2342.610000,1.000000,1310.400000,1310.400000,20487.700000,20453.250000,17.595000
50%,6410.010000,6546.290000,6320.010000,6412.000000,5.270744e+06,2.838960e+05,2717.070000,2717.350000,1.000000,1369.800000,1370.000000,24606.590000,24640.450000,196.340000
75%,9479.285000,9699.530000,9231.755000,9484.275000,2.003407e+07,3.529245e+05,2986.200000,2988.865000,1.000000,1541.300000,1540.400000,26551.095000,26559.540000,365.065000
max,63575.010000,64854.000000,62020.000000,63575.000000,5.606539e+08,4.431451e+06,4185.470000,4175.600000,3.000000,2069.400000,2076.400000,34182.380000,34200.670000,2514.040000


In [27]:
df.dtypes

Open Time             datetime64[ns]
Open                         float64
High                         float64
Low                          float64
Close_btc                    float64
Volume                       float64
NumberTransactions           float64
Close SP500                  float64
Open SP500                   float64
halving                        int64
Close Gold                   float64
Open Gold                    float64
OpenDJ                       float64
CloseDJ                      float64
Close_eth                    float64
dtype: object

In [25]:
df['Open Time'] = pd.to_datetime(df['Open Time'])

In [26]:
print(df)

      Open Time      Open      High       Low  Close_btc         Volume  \
0    2015-10-11    246.30    249.50    246.30     249.50    3021.120000   
1    2015-10-12    249.50    249.50    247.60     247.60    9493.890000   
2    2015-10-13    247.60    252.87    245.75     250.00   12875.400000   
3    2015-10-14    250.00    254.75    250.00     253.00   30115.770000   
4    2015-10-15    253.00    258.98    253.00     255.26   50535.450000   
...         ...       ...       ...       ...        ...            ...   
2014 2021-04-16  63158.74  63520.61  60000.00   61334.80   91764.139884   
2015 2021-04-17  61334.81  62506.05  59580.91   60006.66   58912.256128   
2016 2021-04-18  60006.67  60499.00  50931.30   56150.01  124882.131824   
2017 2021-04-19  56150.01  57526.81  54221.58   55633.14   78229.042267   
2018 2021-04-20  55633.14  56599.53  53329.96   56566.46   60500.152593   

      NumberTransactions  Close SP500  Open SP500  halving  Close Gold  \
0               104550.0 

In [28]:
tipoint.rename(columns={'INTDSRUSM193N':'tipoint'}, inplace=True)

In [29]:
tipoint.describe()

,tipoint
count,64.000000
mean,1.625000
std,0.929029
min,0.250000
25%,1.000000
50%,1.750000
75%,2.312500
max,3.000000


In [30]:
print(tipoint)

          DATE  tipoint  Open Time
0   2015-10-01     0.75 2015-10-01
1   2015-11-01     0.75 2015-11-01
2   2015-12-01     1.00 2015-12-01
3   2016-01-01     1.00 2016-01-01
4   2016-02-01     1.00 2016-02-01
..         ...      ...        ...
59  2020-09-01     0.25 2020-09-01
60  2020-10-01     0.25 2020-10-01
61  2020-11-01     0.25 2020-11-01
62  2020-12-01     0.25 2020-12-01
63  2021-01-01     0.25 2021-01-01

[64 rows x 3 columns]


In [31]:
df_with_tipoint = df.merge(tipoint[["tipoint", "Open Time"]], how='left', on="Open Time")

In [32]:
df_with_tipoint.dtypes

Open Time             datetime64[ns]
Open                         float64
High                         float64
Low                          float64
Close_btc                    float64
Volume                       float64
NumberTransactions           float64
Close SP500                  float64
Open SP500                   float64
halving                        int64
Close Gold                   float64
Open Gold                    float64
OpenDJ                       float64
CloseDJ                      float64
Close_eth                    float64
tipoint                      float64
dtype: object

In [33]:

print(df_with_tipoint)

      Open Time      Open      High       Low  Close_btc         Volume  \
0    2015-10-11    246.30    249.50    246.30     249.50    3021.120000   
1    2015-10-12    249.50    249.50    247.60     247.60    9493.890000   
2    2015-10-13    247.60    252.87    245.75     250.00   12875.400000   
3    2015-10-14    250.00    254.75    250.00     253.00   30115.770000   
4    2015-10-15    253.00    258.98    253.00     255.26   50535.450000   
...         ...       ...       ...       ...        ...            ...   
2014 2021-04-16  63158.74  63520.61  60000.00   61334.80   91764.139884   
2015 2021-04-17  61334.81  62506.05  59580.91   60006.66   58912.256128   
2016 2021-04-18  60006.67  60499.00  50931.30   56150.01  124882.131824   
2017 2021-04-19  56150.01  57526.81  54221.58   55633.14   78229.042267   
2018 2021-04-20  55633.14  56599.53  53329.96   56566.46   60500.152593   

      NumberTransactions  Close SP500  Open SP500  halving  Close Gold  \
0               104550.0 

In [36]:
df_with_tipoint.isna().sum()

Open Time              0
Open                   0
High                   0
Low                    0
Close_btc              0
Volume                 0
NumberTransactions     0
Close SP500            0
Open SP500             0
halving                0
Close Gold             0
Open Gold              0
OpenDJ                 0
CloseDJ                0
Close_eth              0
tipoint               21
dtype: int64

In [35]:
df_with_tipoint.fillna(method='ffill', inplace=True)

In [38]:
print(df_with_tipoint)

      Open Time      Open      High       Low  Close_btc         Volume  \
0    2015-10-11    246.30    249.50    246.30     249.50    3021.120000   
1    2015-10-12    249.50    249.50    247.60     247.60    9493.890000   
2    2015-10-13    247.60    252.87    245.75     250.00   12875.400000   
3    2015-10-14    250.00    254.75    250.00     253.00   30115.770000   
4    2015-10-15    253.00    258.98    253.00     255.26   50535.450000   
...         ...       ...       ...       ...        ...            ...   
2014 2021-04-16  63158.74  63520.61  60000.00   61334.80   91764.139884   
2015 2021-04-17  61334.81  62506.05  59580.91   60006.66   58912.256128   
2016 2021-04-18  60006.67  60499.00  50931.30   56150.01  124882.131824   
2017 2021-04-19  56150.01  57526.81  54221.58   55633.14   78229.042267   
2018 2021-04-20  55633.14  56599.53  53329.96   56566.46   60500.152593   

      NumberTransactions  Close SP500  Open SP500  halving  Close Gold  \
0               104550.0 

In [37]:
df_with_tipoint[df_with_tipoint.isna()]

,Open Time,Open,High,Low,Close_btc,Volume,NumberTransactions,Close SP500,Open SP500,halving,Close Gold,Open Gold,OpenDJ,CloseDJ,Close_eth,tipoint
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df_with_tipoint.fillna(method='bfill', inplace=True)


In [47]:
df_with_tipoint.isna().sum()

Open Time             0
Open                  0
High                  0
Low                   0
Close_btc             0
Volume                0
NumberTransactions    0
Close SP500           0
Open SP500            0
halving               0
Close Gold            0
Open Gold             0
OpenDJ                0
CloseDJ               0
Close_eth             0
tipoint               0
dtype: int64

In [48]:
print(df_with_tipoint)

      Open Time      Open      High       Low  Close_btc         Volume  \
0    2015-10-11    246.30    249.50    246.30     249.50    3021.120000   
1    2015-10-12    249.50    249.50    247.60     247.60    9493.890000   
2    2015-10-13    247.60    252.87    245.75     250.00   12875.400000   
3    2015-10-14    250.00    254.75    250.00     253.00   30115.770000   
4    2015-10-15    253.00    258.98    253.00     255.26   50535.450000   
...         ...       ...       ...       ...        ...            ...   
2014 2021-04-16  63158.74  63520.61  60000.00   61334.80   91764.139884   
2015 2021-04-17  61334.81  62506.05  59580.91   60006.66   58912.256128   
2016 2021-04-18  60006.67  60499.00  50931.30   56150.01  124882.131824   
2017 2021-04-19  56150.01  57526.81  54221.58   55633.14   78229.042267   
2018 2021-04-20  55633.14  56599.53  53329.96   56566.46   60500.152593   

      NumberTransactions  Close SP500  Open SP500  halving  Close Gold  \
0               104550.0 

In [49]:
df_with_tipoint.to_csv('df_with_tipoint.csv')

In [149]:
df_V2 = pd.read_excel('df_excel_V2.xlsx', index_col=0)

In [150]:
df_V2

,Unnamed: 0.1,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,...,RSI_200,CCI_200,MA_300,EMA_300,Momentum_300,BollingerB_300,Bollinger%b_300,S0k_300,RSI_300,CCI_300
0,0,2015-10-11,246.30,249.50,246.30,249.50,3021.120000,104550,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,2015-10-12,249.50,249.50,247.60,247.60,9493.890000,134410,2052.23,2047.93,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.000000,0.000000
2,2,2015-10-13,247.60,252.87,245.75,250.00,12875.400000,134878,2003.69,2015.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,2015-10-14,250.00,254.75,250.00,253.00,30115.770000,141985,1994.24,2003.66,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.596910,0.000000,0.000000
4,4,2015-10-15,253.00,258.98,253.00,255.26,50535.450000,153495,2023.86,1996.47,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.631579,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,2014,2021-04-16,63158.74,63520.61,60000.00,61334.80,91764.139884,2639940,4185.47,4174.14,...,0.500318,1.763313,25307.566700,31286.566728,53849.75,2.784073,1.037233,0.640438,0.503987,2.155158
2015,2015,2021-04-17,61334.81,62506.05,59580.91,60006.66,58912.256128,2268816,4185.47,4174.14,...,0.500318,1.663905,25480.819533,31486.223056,51975.85,2.780519,1.006056,0.379139,0.503987,2.054954
2016,2016,2021-04-18,60006.67,60499.00,50931.30,56150.01,124882.131824,3444089,4185.47,4174.14,...,0.500318,1.592564,25649.859433,31675.727896,50711.97,2.775728,0.982559,0.145549,0.503987,1.982619
2017,2017,2021-04-19,56150.01,57526.81,54221.58,55633.14,78229.042267,2390527,4162.80,4175.60,...,0.500318,1.300518,25804.740500,31838.347946,46464.32,2.768782,0.924720,0.545451,0.503987,1.695163


In [146]:
df_V2.head(205)

,Unnamed: 0.1,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,...,RSI_200,CCI_200,MA_300,EMA_300,Momentum_300,BollingerB_300,Bollinger%b_300,S0k_300,RSI_300,CCI_300
0,0,2015-10-01,246.30,249.50,246.30,249.50,3021.12,104550,0.00,0.00,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,1,2016-01-01,249.50,249.50,247.60,247.60,9493.89,134410,2052.23,2047.93,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0
2,2,2016-04-01,247.60,252.87,245.75,250.00,12875.40,134878,2003.69,2015.00,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,3,2016-07-01,250.00,254.75,250.00,253.00,30115.77,141985,1994.24,2003.66,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.596910,0.0,0.0
4,4,2016-10-01,253.00,258.98,253.00,255.26,50535.45,153495,2023.86,1996.47,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.631579,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,200,NaT,448.44,448.99,435.72,448.99,901276.78,247957,2075.81,2090.93,...,0.685608,0.912672,0.0,0.0,0.0,0.0,0.0,0.063144,0.0,0.0
201,201,NaT,448.99,456.98,448.83,455.91,651149.45,261884,2065.30,2071.82,...,0.685608,0.748688,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0
202,202,NaT,455.91,456.78,448.24,450.30,131025.36,215712,2065.30,2071.82,...,0.674980,0.886014,0.0,0.0,0.0,0.0,0.0,0.868712,0.0,0.0
203,203,NaT,450.30,453.63,448.26,452.66,156471.22,191430,2016.71,2013.78,...,0.674980,0.846948,0.0,0.0,0.0,0.0,0.0,0.241218,0.0,0.0


In [96]:
df_V2.describe()

,Unnamed: 0.1,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,halving,...,RSI_200,CCI_200,MA_300,EMA_300,Momentum_300,BollingerB_300,Bollinger%b_300,S0k_300,RSI_300,CCI_300
count,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2.019000e+03,2.019000e+03,2019.000000,2019.000000,2019.000000,...,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000
mean,1009.000000,8118.617256,8373.885953,7845.329911,8146.388430,1.738177e+07,4.597203e+05,2721.572779,2722.471773,1.340267,...,0.453164,0.804064,5459.530428,5724.366273,3789.273210,1.235777,0.647061,0.580535,0.425068,0.886094
std,582.979416,10787.222476,11161.188328,10387.891407,10839.617374,3.437011e+07,5.251333e+05,509.906876,511.101673,0.752675,...,0.160617,1.498009,4740.267832,5374.746642,9761.444269,0.859003,0.448007,0.269354,0.183269,1.476583
min,0.000000,246.300000,249.500000,236.000000,247.600000,0.000000e+00,1.045500e+05,0.000000,0.000000,0.000000,...,0.000000,-3.496926,0.000000,0.000000,-13297.770000,0.000000,-0.282537,0.000000,0.000000,-2.931247
25%,504.500000,1049.880000,1082.560000,1022.655000,1054.860000,3.166145e+05,2.237780e+05,2341.890000,2342.610000,1.000000,...,0.443563,-0.206737,706.886750,752.071034,0.000000,0.708860,0.275247,0.361690,0.427120,0.000000
50%,1009.000000,6410.010000,6546.290000,6320.010000,6412.000000,5.270744e+06,2.838960e+05,2717.070000,2717.350000,1.000000,...,0.499708,0.773222,5938.586767,6336.365397,1057.960000,1.087076,0.686656,0.609790,0.494637,0.759409
75%,1513.500000,9479.285000,9699.530000,9231.755000,9484.275000,2.003407e+07,3.529245e+05,2986.200000,2988.865000,1.000000,...,0.538249,1.935668,8546.156050,8035.231990,4501.185000,1.785808,1.015587,0.810133,0.532760,2.088887
max,2018.000000,63575.010000,64854.000000,62020.000000,63575.000000,5.606539e+08,4.431451e+06,4185.470000,4175.600000,3.000000,...,0.687505,5.087566,25958.101333,31996.453103,54109.860000,3.613338,1.848980,1.000000,0.629368,5.005236


In [153]:
df_V2.dtypes

Unnamed: 0.1                int64
Open Time          datetime64[ns]
Open                      float64
High                      float64
Low                       float64
                        ...      
BollingerB_300            float64
Bollinger%b_300           float64
S0k_300                   float64
RSI_300                   float64
CCI_300                   float64
Length: 105, dtype: object

In [152]:
df_V2['Open Time'] = pd.to_datetime(df_V2['Open Time'])

In [156]:
pib_usa = pd.read_csv('pib usa 2015 a 2020.csv')

In [157]:
pib_usa

,DATE,A939RX0Q048SBEA
0,2015-10-11,54420.0
1,2016-01-01,54640.0
2,2016-04-01,54722.0
3,2016-07-01,54918.0
4,2016-10-01,55165.0
5,2017-01-01,55401.0
6,2017-04-01,55560.0
7,2017-07-01,55874.0
8,2017-10-01,56324.0
9,2018-01-01,56785.0


In [158]:
pib_usa.describe()

,A939RX0Q048SBEA
count,22.000000
mean,56279.863636
std,1502.653535
min,52448.000000
25%,55224.000000
50%,56554.500000
75%,57401.000000
max,58490.000000


In [161]:
pib_usa.dtypes

DATE               datetime64[ns]
A939RX0Q048SBEA           float64
dtype: object

In [160]:
pib_usa['DATE'] = pd.to_datetime(pib_usa['DATE'])

In [162]:
pib_usa.dtypes

DATE               datetime64[ns]
A939RX0Q048SBEA           float64
dtype: object

In [163]:
pib_usa.rename(columns={'A939RX0Q048SBEA':'PIB'}, inplace=True)

In [164]:
pib_usa.rename(columns={'DATE':'Open Time'}, inplace=True)

In [165]:
pib_usa.rename(columns={'DATE':'Open Time'}, inplace=True)

In [166]:
pib_usa.describe()


,PIB
count,22.000000
mean,56279.863636
std,1502.653535
min,52448.000000
25%,55224.000000
50%,56554.500000
75%,57401.000000
max,58490.000000


In [167]:
pib_usa

,Open Time,PIB
0,2015-10-11,54420.0
1,2016-01-01,54640.0
2,2016-04-01,54722.0
3,2016-07-01,54918.0
4,2016-10-01,55165.0
5,2017-01-01,55401.0
6,2017-04-01,55560.0
7,2017-07-01,55874.0
8,2017-10-01,56324.0
9,2018-01-01,56785.0


In [168]:
pib_usa.describe()


,PIB
count,22.000000
mean,56279.863636
std,1502.653535
min,52448.000000
25%,55224.000000
50%,56554.500000
75%,57401.000000
max,58490.000000


In [75]:
pib_usa

,Open Time,PIB
0,2015-10-01,54420.0
1,2016-01-01,54640.0
2,2016-04-01,54722.0
3,2016-07-01,54918.0
4,2016-10-01,55165.0
5,2017-01-01,55401.0
6,2017-04-01,55560.0
7,2017-07-01,55874.0
8,2017-10-01,56324.0
9,2018-01-01,56785.0


In [169]:
df_V2.dtypes

Unnamed: 0.1                int64
Open Time          datetime64[ns]
Open                      float64
High                      float64
Low                       float64
                        ...      
BollingerB_300            float64
Bollinger%b_300           float64
S0k_300                   float64
RSI_300                   float64
CCI_300                   float64
Length: 105, dtype: object

In [170]:
df_V2 = pd.merge(left=df_V2, right=pib_usa, how='left', left_on='Open Time', right_on='Open Time')
df_V2

,Unnamed: 0.1,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,...,CCI_200,MA_300,EMA_300,Momentum_300,BollingerB_300,Bollinger%b_300,S0k_300,RSI_300,CCI_300,PIB
0,0,2015-10-11,246.30,249.50,246.30,249.50,3021.120000,104550,0.00,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,54420.0
1,1,2015-10-12,249.50,249.50,247.60,247.60,9493.890000,134410,2052.23,2047.93,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.000000,0.000000,NaN
2,2,2015-10-13,247.60,252.87,245.75,250.00,12875.400000,134878,2003.69,2015.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
3,3,2015-10-14,250.00,254.75,250.00,253.00,30115.770000,141985,1994.24,2003.66,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.596910,0.000000,0.000000,NaN
4,4,2015-10-15,253.00,258.98,253.00,255.26,50535.450000,153495,2023.86,1996.47,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.631579,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,2014,2021-04-16,63158.74,63520.61,60000.00,61334.80,91764.139884,2639940,4185.47,4174.14,...,1.763313,25307.566700,31286.566728,53849.75,2.784073,1.037233,0.640438,0.503987,2.155158,NaN
2015,2015,2021-04-17,61334.81,62506.05,59580.91,60006.66,58912.256128,2268816,4185.47,4174.14,...,1.663905,25480.819533,31486.223056,51975.85,2.780519,1.006056,0.379139,0.503987,2.054954,NaN
2016,2016,2021-04-18,60006.67,60499.00,50931.30,56150.01,124882.131824,3444089,4185.47,4174.14,...,1.592564,25649.859433,31675.727896,50711.97,2.775728,0.982559,0.145549,0.503987,1.982619,NaN
2017,2017,2021-04-19,56150.01,57526.81,54221.58,55633.14,78229.042267,2390527,4162.80,4175.60,...,1.300518,25804.740500,31838.347946,46464.32,2.768782,0.924720,0.545451,0.503987,1.695163,NaN


In [171]:
df_V2.fillna(method='ffill', inplace=True)

In [172]:
df_V2

,Unnamed: 0.1,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,...,CCI_200,MA_300,EMA_300,Momentum_300,BollingerB_300,Bollinger%b_300,S0k_300,RSI_300,CCI_300,PIB
0,0,2015-10-11,246.30,249.50,246.30,249.50,3021.120000,104550,0.00,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,54420.0
1,1,2015-10-12,249.50,249.50,247.60,247.60,9493.890000,134410,2052.23,2047.93,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.000000,0.000000,54420.0
2,2,2015-10-13,247.60,252.87,245.75,250.00,12875.400000,134878,2003.69,2015.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,54420.0
3,3,2015-10-14,250.00,254.75,250.00,253.00,30115.770000,141985,1994.24,2003.66,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.596910,0.000000,0.000000,54420.0
4,4,2015-10-15,253.00,258.98,253.00,255.26,50535.450000,153495,2023.86,1996.47,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.631579,0.000000,0.000000,54420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,2014,2021-04-16,63158.74,63520.61,60000.00,61334.80,91764.139884,2639940,4185.47,4174.14,...,1.763313,25307.566700,31286.566728,53849.75,2.784073,1.037233,0.640438,0.503987,2.155158,56812.0
2015,2015,2021-04-17,61334.81,62506.05,59580.91,60006.66,58912.256128,2268816,4185.47,4174.14,...,1.663905,25480.819533,31486.223056,51975.85,2.780519,1.006056,0.379139,0.503987,2.054954,56812.0
2016,2016,2021-04-18,60006.67,60499.00,50931.30,56150.01,124882.131824,3444089,4185.47,4174.14,...,1.592564,25649.859433,31675.727896,50711.97,2.775728,0.982559,0.145549,0.503987,1.982619,56812.0
2017,2017,2021-04-19,56150.01,57526.81,54221.58,55633.14,78229.042267,2390527,4162.80,4175.60,...,1.300518,25804.740500,31838.347946,46464.32,2.768782,0.924720,0.545451,0.503987,1.695163,56812.0


In [173]:
df_V2.describe()


,Unnamed: 0.1,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,halving,...,CCI_200,MA_300,EMA_300,Momentum_300,BollingerB_300,Bollinger%b_300,S0k_300,RSI_300,CCI_300,PIB
count,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2.019000e+03,2.019000e+03,2019.000000,2019.000000,2019.000000,...,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.000000,2019.00000
mean,1009.000000,8118.617256,8373.885953,7845.329911,8146.388430,1.738177e+07,4.597203e+05,2721.572779,2722.471773,1.340267,...,0.804064,5459.530428,5724.366273,3789.273210,1.235777,0.647061,0.580535,0.425068,0.886094,56294.60525
std,582.979416,10787.222476,11161.188328,10387.891407,10839.617374,3.437011e+07,5.251333e+05,509.906876,511.101673,0.752675,...,1.498009,4740.267832,5374.746642,9761.444269,0.859003,0.448007,0.269354,0.183269,1.476583,1459.75212
min,0.000000,246.300000,249.500000,236.000000,247.600000,0.000000e+00,1.045500e+05,0.000000,0.000000,0.000000,...,-3.496926,0.000000,0.000000,-13297.770000,0.000000,-0.282537,0.000000,0.000000,-2.931247,52448.00000
25%,504.500000,1049.880000,1082.560000,1022.655000,1054.860000,3.166145e+05,2.237780e+05,2341.890000,2342.610000,1.000000,...,-0.206737,706.886750,752.071034,0.000000,0.708860,0.275247,0.361690,0.427120,0.000000,55165.00000
50%,1009.000000,6410.010000,6546.290000,6320.010000,6412.000000,5.270744e+06,2.838960e+05,2717.070000,2717.350000,1.000000,...,0.773222,5938.586767,6336.365397,1057.960000,1.087076,0.686656,0.609790,0.494637,0.759409,56785.00000
75%,1513.500000,9479.285000,9699.530000,9231.755000,9484.275000,2.003407e+07,3.529245e+05,2986.200000,2988.865000,1.000000,...,1.935668,8546.156050,8035.231990,4501.185000,1.785808,1.015587,0.810133,0.532760,2.088887,57429.00000
max,2018.000000,63575.010000,64854.000000,62020.000000,63575.000000,5.606539e+08,4.431451e+06,4185.470000,4175.600000,3.000000,...,5.087566,25958.101333,31996.453103,54109.860000,3.613338,1.848980,1.000000,0.629368,5.005236,58490.00000


In [175]:
df_V2.isna().sum()

Unnamed: 0.1       0
Open Time          0
Open               0
High               0
Low                0
                  ..
Bollinger%b_300    0
S0k_300            0
RSI_300            0
CCI_300            0
PIB                0
Length: 106, dtype: int64

In [176]:
cpi = pd.read_csv ('cpi.csv')

In [177]:
cpi

,DATE,CPALTT01USM657N
0,2015-10-11,-0.044968
1,2015-11-01,-0.211068
2,2015-12-01,-0.341710
3,2016-01-01,0.165310
4,2016-02-01,0.082308
...,...,...
60,2020-10-01,0.041494
61,2020-11-01,-0.061063
62,2020-12-01,0.094148
63,2021-01-01,0.425378


In [178]:
cpi.describe()

,CPALTT01USM657N
count,65.000000
mean,0.154562
std,0.262458
min,-0.668694
25%,0.001950
50%,0.139275
75%,0.387977
max,0.582773


In [179]:
cpi.dtypes


DATE                object
CPALTT01USM657N    float64
dtype: object

In [180]:
cpi.rename(columns={'CPALTT01USM657N':'CPI'}, inplace=True)

In [181]:
cpi['DATE'] = pd.to_datetime(cpi['DATE'])

In [182]:
cpi.rename(columns={'DATE':'Open Time'}, inplace=True)

In [183]:
cpi

,Open Time,CPI
0,2015-10-11,-0.044968
1,2015-11-01,-0.211068
2,2015-12-01,-0.341710
3,2016-01-01,0.165310
4,2016-02-01,0.082308
...,...,...
60,2020-10-01,0.041494
61,2020-11-01,-0.061063
62,2020-12-01,0.094148
63,2021-01-01,0.425378


In [184]:
print(cpi)

    Open Time       CPI
0  2015-10-11 -0.044968
1  2015-11-01 -0.211068
2  2015-12-01 -0.341710
3  2016-01-01  0.165310
4  2016-02-01  0.082308
..        ...       ...
60 2020-10-01  0.041494
61 2020-11-01 -0.061063
62 2020-12-01  0.094148
63 2021-01-01  0.425378
64 2021-02-01  0.547438

[65 rows x 2 columns]


In [185]:
df_V2 = pd.merge(left=df_V2, right=cpi, how='left', left_on='Open Time', right_on='Open Time')
df_V2

,Unnamed: 0.1,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,...,MA_300,EMA_300,Momentum_300,BollingerB_300,Bollinger%b_300,S0k_300,RSI_300,CCI_300,PIB,CPI
0,0,2015-10-11,246.30,249.50,246.30,249.50,3021.120000,104550,0.00,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,54420.0,-0.044968
1,1,2015-10-12,249.50,249.50,247.60,247.60,9493.890000,134410,2052.23,2047.93,...,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.000000,0.000000,54420.0,NaN
2,2,2015-10-13,247.60,252.87,245.75,250.00,12875.400000,134878,2003.69,2015.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,54420.0,NaN
3,3,2015-10-14,250.00,254.75,250.00,253.00,30115.770000,141985,1994.24,2003.66,...,0.000000,0.000000,0.00,0.000000,0.000000,0.596910,0.000000,0.000000,54420.0,NaN
4,4,2015-10-15,253.00,258.98,253.00,255.26,50535.450000,153495,2023.86,1996.47,...,0.000000,0.000000,0.00,0.000000,0.000000,0.631579,0.000000,0.000000,54420.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,2014,2021-04-16,63158.74,63520.61,60000.00,61334.80,91764.139884,2639940,4185.47,4174.14,...,25307.566700,31286.566728,53849.75,2.784073,1.037233,0.640438,0.503987,2.155158,56812.0,NaN
2015,2015,2021-04-17,61334.81,62506.05,59580.91,60006.66,58912.256128,2268816,4185.47,4174.14,...,25480.819533,31486.223056,51975.85,2.780519,1.006056,0.379139,0.503987,2.054954,56812.0,NaN
2016,2016,2021-04-18,60006.67,60499.00,50931.30,56150.01,124882.131824,3444089,4185.47,4174.14,...,25649.859433,31675.727896,50711.97,2.775728,0.982559,0.145549,0.503987,1.982619,56812.0,NaN
2017,2017,2021-04-19,56150.01,57526.81,54221.58,55633.14,78229.042267,2390527,4162.80,4175.60,...,25804.740500,31838.347946,46464.32,2.768782,0.924720,0.545451,0.503987,1.695163,56812.0,NaN


In [187]:
df_V2.to_csv('df_V3.csv')

In [196]:
df_V2.isna().sum()


Unnamed: 0.1    0
Open Time       0
Open            0
High            0
Low             0
               ..
S0k_300         0
RSI_300         0
CCI_300         0
PIB             0
CPI             0
Length: 107, dtype: int64

In [194]:
df_V2.fillna(method='ffill', inplace=True)

In [195]:
df_V2


,Unnamed: 0.1,Open Time,Open,High,Low,Close,Volume,NumberTransactions,Close SP500,Open SP500,...,MA_300,EMA_300,Momentum_300,BollingerB_300,Bollinger%b_300,S0k_300,RSI_300,CCI_300,PIB,CPI
0,0,2015-10-11,246.30,249.50,246.30,249.50,3021.120000,104550,0.00,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,54420.0,-0.044968
1,1,2015-10-12,249.50,249.50,247.60,247.60,9493.890000,134410,2052.23,2047.93,...,0.000000,0.000000,0.00,0.000000,0.000000,1.000000,0.000000,0.000000,54420.0,-0.044968
2,2,2015-10-13,247.60,252.87,245.75,250.00,12875.400000,134878,2003.69,2015.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,54420.0,-0.044968
3,3,2015-10-14,250.00,254.75,250.00,253.00,30115.770000,141985,1994.24,2003.66,...,0.000000,0.000000,0.00,0.000000,0.000000,0.596910,0.000000,0.000000,54420.0,-0.044968
4,4,2015-10-15,253.00,258.98,253.00,255.26,50535.450000,153495,2023.86,1996.47,...,0.000000,0.000000,0.00,0.000000,0.000000,0.631579,0.000000,0.000000,54420.0,-0.044968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,2014,2021-04-16,63158.74,63520.61,60000.00,61334.80,91764.139884,2639940,4185.47,4174.14,...,25307.566700,31286.566728,53849.75,2.784073,1.037233,0.640438,0.503987,2.155158,56812.0,0.547438
2015,2015,2021-04-17,61334.81,62506.05,59580.91,60006.66,58912.256128,2268816,4185.47,4174.14,...,25480.819533,31486.223056,51975.85,2.780519,1.006056,0.379139,0.503987,2.054954,56812.0,0.547438
2016,2016,2021-04-18,60006.67,60499.00,50931.30,56150.01,124882.131824,3444089,4185.47,4174.14,...,25649.859433,31675.727896,50711.97,2.775728,0.982559,0.145549,0.503987,1.982619,56812.0,0.547438
2017,2017,2021-04-19,56150.01,57526.81,54221.58,55633.14,78229.042267,2390527,4162.80,4175.60,...,25804.740500,31838.347946,46464.32,2.768782,0.924720,0.545451,0.503987,1.695163,56812.0,0.547438


In [ ]:
print(cpi)